In [1]:
import os
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

e:\conda\envs\summa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
persist_directory="dbbbb"

In [4]:
vectordb1=Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

#Retrieval operation
retriever = vectordb1.as_retriever()

In [5]:
model = ChatOpenAI(openai_api_key=os.getenv("API_KEY"))

e:\conda\envs\summa\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
#The Template for the OpenAI model to understand and generate answers. Given from the system
template = """You are an assistant for question-answering tasks. \
Answer the question based only on the following context: \
Summarize and answer \
Based on the given tweet context give the sentiment of the tweet, name of the stock affected. Give an one line explanation for the reason of affection. \

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template) #prompt for the OpenAI model

In [7]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} #retrieving
    | prompt
    | model #generating
    | StrOutputParser()
)

In [9]:
print(chain.invoke("apple watch is so bad to use")) #User query

The sentiment of the tweet is negative. The stock affected is AAPL (Apple Inc.). The reason for the negative sentiment is the dissatisfaction with the use of the Apple Watch.
